<h3>section # 5

In <b>object detection</b>, we detect an object in a frame, put a bounding box or a mask around it and classify the object. Note that, the job of the detector ends here. It processes each frame independently and identifies numerous objects in that particular frame.

Now, an object tracker on the other hand needs to track a particular object across the entire video. If the detector detects 3 cars in the frame, the object tracker has to identify the 3 separate detections and needs to track it across the subsequent frames (with the help of a unique ID).

<b>Object detection </b>is simply about identifying and locating all known objects in a scene.

<b>Object tracking </b>is about locking onto a particular moving object(s) in real-time.

The two are similar, however.

Object detection can occur on still photos while object tracking needs video feed.<br>
Object detection can be used as object tracking if we run object detection every frame per second.<br>
Object tracking need not identify the objects. Objects can be tracked based solely on motion features without knowing the actual objects being tracked. Thus pure object tracking can be extremely efficient if it leverages the temporal relationship between video frames.<br>
Running object detection as tracking can be computationally expensive and it can only track known objects.<br>
Thus object detection requires a form of classification and localization.<br>

While for object tracking, classification can be unnecessary depending on the problem.<br>

In practice, real-time object detection can be sped up by employing object tracking and running classification once a few frames. Object detection can run on a slow thread looking for objects to lock onto and once those objects are locked on then object tracking, running in a faster thread, can takeover.



In [1]:
#Object Tracking ...
import cv2
import numpy as np

#cap = cv2.VideoCapture('Desktop/vtest.avi')
cap = cv2.VideoCapture(0)

frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
out = cv2.VideoWriter("output.avi", fourcc, 5.0, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()
##print(frame1.shape)
while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        print(f'value of x is {x} and y is {y} and w is {w} and h is {h}')

        if cv2.contourArea(contour) < 300: 
            continue
        
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        #cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)

    image = cv2.resize(frame1, (1280,720))
    out.write(image)
    cv2.imshow("feed", frame1)
    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()
out.release()

#######################fgg


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import cv2 
cap = cv2.VideoCapture('Desktop/vtest.avi')

object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)
while True:
    ret, frame = cap.read()
    #frame = cv2.resize(frame, (800,600))
    height, width, _ = frame.shape
    
    # Extract Region of interst    
    # 100 200   150 200    
    roi = frame[0:400, 100:312]
    # roi = frame[100:250, 300:400]
    # roi = frame[500:600, 300:400]

    # object detecti    on 
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE )
    for cnt in contours:
        # calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            # cv2.drawContours(roi, [cnt], -1, (0,255,0), 2)
            x, y, h, w = cv2.boundingRect(cnt)
            cv2.rectangle(roi, (x, y), (x+ y, w+h), (0,255,0), 3)

    cv2.imshow('roi',roi)
    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)

    key = cv2.waitKey(20)
    if  key == 27 :
        break

cap.release()
cv2.destroyAllWindows()

<b>Background subtraction </b>is a mainstream algorithm for moving object detection in video surveillance systems. It segments moving objects by using the difference between the background and input images. The key to background subtraction is to establish a reliable initial background.

In [24]:
# in this example , we are going to build object detecor using background subtractor
import cv2 
cap = cv2.VideoCapture('Desktop/cars.mp4')
# cap = cv2.VideoCapture('Desktop/vtest.avi')
# object detector
object_detector=cv2.createBackgroundSubtractorMOG2()#history=100,varThreshold=120
# object_detector=cv2.createBackgroundSubtractorKNN()
while cap.isOpened():
    ret , frame= cap.read()
    #print(frame.shape[0],frame.shape[1]) 336,596
    extractedROI=frame[50:363,363:596]
    #apply background subtractor
    mask=object_detector.apply(extractedROI)
#     _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours,_=cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #     print(contours)
    #     print(mask)
    
    for cnt in contours:
#         cv2.drawContours(frame,cnt,-1,(255,0,0),3) # contours everwhere!!!!
        # sol:
        contourArea=cv2.contourArea(cnt)
        if(contourArea <80):            
#             cv2.drawContours(extractedROI,[cnt],-1,(255,0,0),3) # contours everwhere!!!!
            x,y,w,h=cv2.boundingRect(cnt)
            cv2.rectangle(extractedROI,(x,y),(x + w , y + h) , (0,255,0),2)
    cv2.imshow("Frame", frame)
    cv2.imshow("mask", mask)
    cv2.imshow('ROI',extractedROI)

    myKey=cv2.waitKey(30)
    if myKey== 27:
        break
cap.release()
cv2.destroyAllWindows()

In [18]:
# in this example , we are going to build object detecor using background subtractor
import cv2 
cap = cv2.VideoCapture('Desktop/cars.mp4')
# cap = cv2.VideoCapture('Desktop/vtest.avi')

# object detector
object_detector=cv2.createBackgroundSubtractorMOG2()#history=100,varThreshold=120
# object_detector=cv2.createBackgroundSubtractorKNN()
while cap.isOpened():
    ret , frame= cap.read()
    #print(frame.shape[0],frame.shape[1]) 336,596
    extractedROI=frame[50:363,363:596]
    #apply background subtractor
    mask=object_detector.apply(extractedROI)
#     _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours,_=cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #     print(contours)
    #     print(mask)
    
    for cnt in contours:
#         cv2.drawContours(frame,cnt,-1,(255,0,0),3) # contours everwhere!!!!
        # sol:
        contourArea=cv2.contourArea(cnt)
        if(contourArea <200):            
#             cv2.drawContours(extractedROI,[cnt],-1,(255,0,0),3) # contours everwhere!!!!
            x,y,w,h=cv2.boundingRect(cnt)
            print(x,y,w,h)
            cv2.rectangle(extractedROI,(x,y),(x + w , y + h) , (0,255,0),2)
    cv2.imshow("Frame", frame)
    cv2.imshow("mask", mask)
    cv2.imshow('ROI',extractedROI)

    myKey=cv2.waitKey(30)
    if myKey== 27:
        break
cap.release()
cv2.destroyAllWindows()

146 259 6 2
153 258 1 2
229 247 4 4
225 252 3 3
204 238 29 3
227 235 1 1
222 235 3 1
189 233 1 1
227 229 3 1
211 229 1 1
209 229 1 1
201 228 2 2
214 227 12 2
173 226 2 12
171 225 1 8
224 223 1 1
174 223 5 5
225 221 2 1
216 220 8 3
220 220 3 3
217 219 1 1
215 219 1 1
223 217 10 9
202 218 3 3
199 218 4 3
198 217 3 3
187 217 3 4
193 213 3 3
194 212 4 3
186 210 3 3
175 210 3 3
172 209 3 3
184 207 3 3
174 205 3 3
182 204 3 3
177 203 3 6
167 203 8 6
173 202 3 3
178 201 3 4
179 199 4 10
182 198 3 3
180 195 3 3
214 193 6 4
181 193 5 6
228 192 3 3
216 192 14 4
198 192 4 3
191 192 10 5
184 192 3 3
187 190 3 3
220 189 6 3
188 189 5 3
118 173 2 1
142 169 32 7
208 165 1 1
184 161 1 1
135 160 1 1
133 158 1 1
157 154 8 1
163 150 1 1
173 149 1 1
157 149 4 1
124 149 5 15
187 144 1 1
158 144 1 1
149 144 4 1
88 141 1 1
89 137 1 1
218 133 15 2
217 131 16 1
225 129 1 1
194 122 4 7
85 121 1 1
199 120 1 2
212 119 1 1
132 147 4 3
181 146 3 3
145 146 3 3
128 146 3 3
144 145 3 3
129 144 3 3
176 142 4 3
135 139 

131 93 2 1
108 92 6 2
100 92 1 1
136 91 6 3
139 90 2 1
102 90 5 3
153 88 1 2
109 86 1 1
153 85 1 1
134 83 3 4
131 82 1 1
93 82 3 1
152 81 1 1
128 81 1 1
107 80 17 1
191 78 2 11
109 78 1 1
107 78 1 1
181 76 10 4
186 75 1 1
183 75 1 1
121 75 10 4
89 75 31 29
96 100 3 3
95 94 3 3
92 91 5 4
97 88 3 3
91 88 7 5
97 84 3 3
100 82 3 3
98 82 3 3
147 72 2 1
161 71 26 2
188 70 3 2
148 70 2 1
183 67 1 1
75 67 1 1
200 66 1 1
163 66 8 1
176 65 3 2
199 63 1 1
196 63 1 3
157 62 1 1
132 62 1 1
151 60 1 1
132 60 1 1
194 59 1 1
150 59 21 21
161 76 3 3
159 76 3 3
154 75 3 3
152 73 3 3
154 72 3 3
155 69 3 3
156 67 4 5
134 59 1 1
219 58 1 1
198 58 1 1
165 58 4 2
150 58 1 1
79 58 2 1
208 57 5 5
226 56 7 6
228 58 3 3
229 57 4 4
198 56 1 1
169 55 3 2
97 54 1 2
169 53 1 1
70 53 5 3
67 53 2 2
140 52 1 1
130 52 3 2
82 52 11 4
196 51 3 3
165 51 1 1
98 51 1 1
155 49 1 1
231 48 2 3
229 47 1 1
160 47 3 1
95 47 3 3
93 47 2 3
57 47 3 3
225 46 3 1
155 46 4 2
135 46 11 16
137 55 3 3
140 46 4 3
200 45 24 16
200 57 3 3
202

165 10 1 1
136 10 2 1
162 9 1 1
148 9 5 11
102 9 2 1
143 7 2 2
210 6 2 3
186 6 1 1
100 4 1 2
62 4 1 1
55 4 1 1
47 3 4 1
40 3 3 1
150 2 1 1
32 2 4 6
180 1 1 1
28 0 2 2
163 285 2 1
184 284 4 2
185 282 1 1
213 281 7 5
214 282 3 3
215 281 3 3
174 281 1 1
222 280 1 1
164 280 1 1
218 279 3 2
186 279 3 3
221 278 12 8
163 278 1 1
170 274 7 8
164 274 5 4
158 274 5 12
159 282 3 3
151 258 1 1
143 249 5 5
148 247 1 3
151 245 2 2
144 245 2 2
162 244 1 1
200 242 3 1
140 242 1 1
196 241 1 1
145 241 2 1
195 239 1 1
146 239 1 1
221 238 1 1
192 238 1 1
206 237 3 1
230 236 3 1
225 236 4 2
148 236 11 9
181 235 1 1
144 235 2 3
232 234 1 1
228 234 2 1
231 232 2 1
227 232 3 1
228 230 3 1
183 230 38 12
215 237 3 3
165 227 1 4
147 226 9 10
147 225 1 1
223 216 1 1
164 215 1 1
230 214 3 3
221 214 3 1
206 212 2 1
181 212 2 1
209 211 3 1
185 211 1 1
222 207 2 1
207 205 1 1
167 205 1 1
213 204 1 1
149 204 2 1
215 203 2 1
205 203 1 1
203 203 1 1
198 203 3 1
155 203 8 22
209 202 4 4
139 201 5 3
157 198 1 1
118 196 2 

229 109 4 3
227 106 3 3
177 103 3 1
206 102 1 1
204 102 1 1
200 102 1 1
173 102 2 1
86 102 1 1
75 102 1 1
208 101 24 3
165 101 1 1
105 101 2 1
103 101 1 1
92 101 1 1
154 100 4 1
221 99 1 1
136 99 1 1
231 98 1 1
186 97 5 5
116 96 3 4
192 94 11 18
195 107 3 3
194 102 3 3
61 91 1 1
197 90 31 12
196 90 1 3
170 88 2 1
116 88 5 3
176 87 1 1
106 87 7 3
103 87 2 2
100 87 1 1
145 86 3 6
214 85 1 1
118 85 1 1
125 84 12 14
133 92 3 3
76 84 1 1
213 82 3 1
219 81 1 1
84 81 1 2
131 80 1 2
101 79 1 1
146 78 3 5
87 78 3 1
73 78 10 2
71 78 1 1
137 77 1 1
130 77 1 1
69 77 1 1
67 77 1 1
63 77 1 1
58 77 1 1
132 76 2 2
52 76 1 2
187 75 2 9
54 75 1 1
45 75 1 1
60 74 1 1
56 73 2 3
177 72 8 4
45 72 1 1
87 93 3 3
95 91 3 3
89 89 3 3
93 88 4 3
90 84 3 5
88 83 3 3
91 77 8 14
99 74 3 3
98 73 3 3
101 71 25 6
33 71 1 1
85 70 1 1
187 69 1 1
177 69 6 1
165 69 1 1
232 68 1 1
141 68 2 2
48 68 1 1
161 67 1 1
184 66 3 3
145 66 2 2
223 65 1 1
128 65 1 1
191 64 2 3
145 64 1 1
78 64 1 1
70 64 5 1
112 63 2 1
109 62 1 1
207 6

173 18 4 3
131 15 2 1
136 14 2 3
129 14 1 1
76 13 1 1
67 12 1 6
43 12 1 1
155 11 1 1
127 11 2 1
122 11 2 1
106 11 21 12
106 18 3 4
108 13 3 3
76 11 1 1
69 11 1 1
59 11 4 1
56 11 1 1
51 11 1 1
45 11 3 1
164 9 2 1
159 9 1 1
144 9 7 11
147 13 3 3
134 9 2 2
68 9 1 1
32 9 1 1
98 8 4 4
34 8 1 1
141 6 2 4
139 6 1 1
38 6 1 1
211 5 1 1
208 5 2 3
184 5 1 1
146 5 1 1
144 5 1 1
74 5 2 2
155 3 2 1
98 3 2 1
55 3 11 1
53 3 1 1
148 1 1 2
140 1 2 2
53 1 2 1
50 1 1 1
43 1 1 1
206 0 2 1
84 0 1 1
80 0 1 1
31 0 4 7
28 0 4 3
205 284 1 2
199 284 1 2
162 284 1 1
217 283 2 3
169 281 1 1
206 280 5 6
165 280 1 1
162 280 2 2
216 279 1 1
169 278 1 2
162 278 1 1
171 277 1 1
153 276 8 3
176 275 1 1
164 275 1 1
166 274 1 1
168 273 2 2
218 271 4 2
215 270 1 1
220 269 1 1
225 268 3 2
200 268 1 1
152 267 12 7
160 267 3 3
225 266 4 2
230 265 1 2
165 265 9 8
227 263 1 1
228 278 5 4
178 278 4 3
190 277 4 4
227 276 4 3
230 275 3 3
184 275 3 3
193 274 3 3
201 273 3 3
183 273 6 4
208 272 4 3
205 272 3 3
203 271 3 5
198 271 3 

...

196 40 1 1
149 40 2 2
138 40 3 2
91 40 1 1
81 40 6 5
79 40 1 1
130 39 4 2
124 38 1 1
134 37 1 1
54 37 5 7
131 36 1 1
110 36 8 3
45 36 7 6
49 39 3 3
49 36 3 3
232 35 1 1
230 35 1 1
54 35 1 1
179 33 54 22
186 52 3 3
187 49 3 3
193 47 3 3
188 47 3 3
190 46 3 3
189 43 3 3
185 43 3 4
99 32 6 2
52 32 1 1
14 32 1 1
178 31 1 1
18 31 1 1
128 29 12 8
132 33 3 3
77 29 1 1
188 28 1 1
174 28 1 1
168 28 1 1
179 26 4 4
140 26 14 8
146 29 3 3
100 26 1 1
52 26 8 4
184 25 2 5
167 25 6 4
224 24 2 1
124 24 3 3
79 24 19 17
88 35 3 3
79 30 3 3
184 23 1 1
154 23 2 1
108 23 1 1
106 23 1 1
104 23 1 1
227 22 6 4
156 22 13 11
164 28 3 4
223 21 1 1
51 21 1 1
229 20 4 1
97 20 1 1
173 19 9 5
101 19 2 7
166 18 5 1
123 17 1 1
172 16 1 1
119 16 3 1
105 16 1 1
61 16 3 4
51 16 8 4
31 16 13 7
212 15 1 1
107 15 7 1
210 14 23 15
222 26 3 3
227 15 3 3
63 14 1 1
58 14 1 1
24 14 1 2
152 13 11 9
155 18 3 3
154 15 3 3
159 13 3 3
33 13 1 1
214 12 3 2
211 12 2 2
29 12 3 4
159 11 1 1
63 11 1 1
65 10 1 1
37 10 3 2
185 9 2 1
58 9 2 

155 67 25 18
161 80 3 3
158 75 3 3
175 70 3 3
171 70 3 3
110 67 1 1
76 67 1 1
31 67 1 1
212 65 6 3
218 63 1 1
191 63 2 2
111 63 1 1
92 63 3 2
165 62 1 1
89 62 2 2
226 90 3 3
223 86 3 3
201 80 4 3
192 80 3 3
204 79 4 4
199 79 3 3
203 78 3 3
197 78 3 3
194 78 3 3
211 75 3 3
191 68 3 3
167 61 3 2
216 60 1 1
91 60 1 1
85 60 3 3
34 59 1 1
230 58 3 4
216 58 1 1
218 57 1 1
88 57 1 1
23 57 1 1
220 56 13 12
222 57 4 3
140 56 1 1
28 56 1 1
158 54 2 1
224 53 2 2
161 52 5 8
155 52 2 1
27 52 2 2
219 51 1 1
133 50 6 8
92 50 1 1
66 50 44 20
106 63 3 3
102 60 5 3
71 60 3 3
106 57 3 3
105 56 3 3
77 56 3 3
71 56 3 4
102 54 3 3
72 54 3 4
101 53 3 3
89 53 3 3
75 52 3 3
197 49 1 1
165 49 1 1
79 48 2 2
45 48 1 1
98 47 5 3
90 47 1 1
77 47 1 2
32 47 1 1
94 46 1 1
159 45 3 3
207 44 26 12
208 47 3 4
118 43 1 1
86 43 1 1
41 43 4 3
116 57 4 3
125 54 3 3
119 52 3 3
127 51 3 3
117 49 3 3
124 48 3 3
121 48 6 8
118 45 3 3
129 43 3 3
20 42 1 1
17 42 1 1
226 41 6 3
129 41 1 1
122 41 6 3
149 40 2 2
139 40 2 2
91 40 1 1


172 124 3 3
169 124 3 3
174 123 4 4
172 122 4 3
155 121 21 14
171 130 1 1
156 120 3 3
169 119 5 4
162 119 3 3
168 118 3 3
166 115 3 3
155 115 5 3
164 114 3 3
164 112 3 3
161 111 3 3
152 108 1 1
54 108 1 2
207 107 3 1
149 106 2 3
73 106 1 1
50 105 2 2
48 103 1 1
71 101 1 1
51 101 2 1
224 99 1 1
70 99 1 1
137 97 3 5
180 95 1 1
84 94 4 3
221 93 1 1
40 93 1 1
9 93 1 1
111 92 1 1
87 92 1 1
49 92 1 1
3 92 1 1
53 91 1 1
10 91 1 1
57 90 1 1
42 90 1 1
130 88 1 1
55 88 1 1
66 87 1 1
7 86 1 1
129 85 1 1
125 85 1 1
14 85 1 1
203 84 1 1
127 83 14 13
86 83 6 4
17 83 2 1
15 83 1 1
5 83 3 3
38 82 1 1
11 82 1 1
1 81 2 1
178 80 7 14
179 85 3 3
176 80 2 2
91 79 2 1
88 79 1 1
44 79 1 1
1 79 1 1
70 78 1 1
11 78 1 1
101 77 32 8
76 76 1 1
177 75 2 2
92 73 38 23
71 71 1 1
67 70 3 3
178 69 2 2
78 69 1 1
73 69 1 1
155 68 1 1
1 68 1 1
184 67 1 1
109 67 6 5
68 67 1 1
31 67 1 1
207 66 2 1
228 94 4 5
227 93 3 3
226 90 3 3
222 79 3 3
203 78 3 4
202 77 3 3
201 76 3 3
196 76 4 4
217 75 3 3
211 74 3 4
202 74 3 3
212 73

150 131 1 1
31 130 1 1
29 130 1 1
232 129 1 1
229 129 2 1
62 128 1 1
231 127 1 1
63 121 1 1
3 118 1 1
60 117 1 1
66 115 1 2
22 114 1 1
24 113 1 1
18 113 1 1
112 110 16 4
109 110 2 1
66 109 1 1
63 109 2 1
60 109 1 1
202 108 1 1
54 108 1 2
14 108 1 1
207 107 3 1
70 107 1 1
20 107 1 1
202 106 1 1
50 106 2 1
206 103 5 3
71 101 1 1
162 100 1 1
70 99 1 1
156 98 1 1
158 96 3 2
154 96 1 1
186 145 3 6
190 142 3 3
188 142 5 6
178 137 10 10
168 137 3 3
179 135 4 3
169 135 3 3
162 135 3 3
117 135 3 3
159 131 3 3
141 131 3 3
122 130 3 3
120 130 3 3
171 129 3 4
167 129 4 3
116 129 3 3
178 127 3 3
175 125 3 3
171 125 4 5
172 124 3 3
174 123 4 4
172 122 4 3
128 121 3 3
131 120 3 4
169 119 5 4
132 118 3 3
167 116 3 3
133 116 3 3
166 115 3 3
160 102 3 3
147 98 3 3
179 94 2 1
40 93 1 1
9 93 1 1
84 92 3 6
10 91 1 1
56 90 2 1
53 90 2 2
42 90 1 1
138 88 1 1
55 88 1 1
66 87 1 1
103 85 2 1
100 85 2 1
14 85 1 1
93 83 1 1
17 83 3 1
15 83 1 1
5 83 3 4
201 82 1 1
10 82 2 1
1 81 2 1
208 80 1 1
135 80 1 1
107 80 5 

88 142 4 4
114 141 1 1
142 140 2 2
32 140 2 2
111 139 2 3
142 138 1 1
136 138 2 3
222 137 1 1
66 137 1 1
136 136 1 1
108 136 2 3
219 135 1 1
161 135 1 1
224 134 1 1
126 134 3 2
26 134 3 1
1 134 4 1
140 133 1 1
225 132 8 8
218 132 4 2
31 132 2 2
110 131 1 1
215 130 1 1
31 130 1 1
29 130 1 1
232 129 1 1
229 129 2 1
62 128 1 1
231 127 1 1
229 126 1 1
63 121 1 1
3 118 1 1
60 117 1 1
66 115 1 2
22 114 1 1
24 113 1 1
18 113 1 1
109 109 1 1
66 109 1 1
63 109 2 1
54 109 1 1
111 108 16 4
60 108 1 2
13 108 2 2
70 107 1 1
51 106 1 1
11 105 1 1
203 102 6 5
203 103 3 3
224 101 1 1
162 100 1 1
70 99 1 1
61 98 1 1
158 96 3 2
154 96 2 1
84 96 1 2
185 151 3 3
186 145 3 7
181 135 3 3
117 135 3 3
180 134 3 3
171 134 4 4
176 133 12 14
174 133 4 4
180 132 3 3
152 131 3 3
126 130 3 3
167 129 3 3
166 127 3 3
149 127 3 3
171 125 4 4
146 125 3 3
116 125 3 3
129 124 3 3
176 123 3 4
172 122 5 4
123 122 6 10
132 121 4 5
131 121 3 4
170 119 4 4
130 119 3 3
125 117 3 3
167 116 3 4
131 116 3 4
166 115 3 3
132 114 3 

<hr>

<h3>Histogram 

<b>A histogram represents the distribution of pixel intensities</b> (whether color or grayscale) in an image. It can be visualized as a graph (or plot) that gives a high-level intuition of the intensity (pixel value) distribution. We are going to assume a RGB color space in this example, so these pixel values will be in the range of 0 to 255.

When plotting the histogram, the x-axis serves as our “bins.” If we construct <b>a histogram with 256 bins</b>, then we are effectively counting <b>the number of times each pixel value occurs.</b>

<b>cv2.calcHist(images, channels, mask, histSize, ranges)</b>
<li><b>images:</b> This is the image that we want to compute a histogram for. Wrap it as a list: [myImage] .
<li><b>channels:</b> A list of indexes, where we specify the index of the channel we want to compute a histogram for. To compute a histogram of a grayscale image, the list would be [0]. To compute a histogram for all three red, green, and blue channels, the channels list would be [0, 1, 2].
<li><b>mask: </b>Remember learning about masks in my Image Masking with OpenCV guide? Well, here we can supply a mask. If a mask is provided, a histogram will be computed for masked pixels only. If we do not have a mask or do not want to apply one, we can just provide a value of None.
<li><b>histSize:</b> This is the number of bins we want to use when computing a histogram. Again, this is a list, one for each channel we are computing a histogram for. The bin sizes do not all have to be the same. Here is an example of 32 bins for each channel: [32, 32, 32].
<li><b>ranges: </b>The range of possible pixel values. Normally, this is [0, 256] (that is not a typo — the ending range of the cv2.calcHist function is non-inclusive so you’ll want to provide a value of 256 rather than 255) for each channel, but if you are using a color space other than RGB [such as HSV], the ranges might be different.)

In [ ]:
Steps to plot a histogram in Python using Matplotlib
Step 1: Install the Matplotlib package. ...
Step 2: Collect the data for the histogram. ...
Step 3: Determine the number of bins. ...
Step 4: Plot the histogram in Python using matplotlib.

In [13]:
#################################################

###### histogram


import numpy as np 
import cv2
from matplotlib import pyplot as plt

img1 = np.zeros((200,200), np.uint8)
cv2.imshow('',img1)

In [8]:
#cv2.imshow('img', img)
plt.hist(img1.ravel(), 256, [0,256])

plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()


In [15]:
import numpy as np 
import cv2
from matplotlib import pyplot as plt

img1 = np.zeros((200,200), np.uint8)

cv2.rectangle(img1, (0,100), (200,200), 255, -1)

cv2.imshow('img', img1)
plt.hist(img1.ravel(), 256, [0,256])

plt.show()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
import numpy as np 
import cv2
from matplotlib import pyplot as plt

img1 = np.zeros((200,200), np.uint8)

cv2.rectangle(img1, (0,100), (200,200), 255, -1)
cv2.rectangle(img1, (0,50), (100,100), 127, -1)


cv2.imshow('img', img1)
plt.hist(img1.ravel(), 256, [0,256])

plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
import numpy as np 
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('Desktop/lenna.png')
print(cv2.split(img1))
b, g, r = cv2.split(img1)


cv2.imshow('img', img1)
cv2.imshow('b', b)
cv2.imshow('g', g)
cv2.imshow('r', r)

plt.hist(img1.ravel(), 256, [0,256])
plt.hist(b.ravel(), 256, [0,256])
plt.hist(g.ravel(), 256, [0,256])
plt.hist(r.ravel(), 256, [0,256])
 
plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[125, 125, 133, ..., 122, 110,  90],
       [125, 125, 133, ..., 122, 110,  90],
       [125, 125, 133, ..., 122, 110,  90],
       ...,
       [ 60,  60,  58, ...,  84,  76,  79],
       [ 57,  57,  62, ...,  79,  81,  81],
       [ 57,  57,  62, ...,  79,  81,  81]], dtype=uint8), array([[137, 137, 137, ..., 148, 130,  99],
       [137, 137, 137, ..., 148, 130,  99],
       [137, 137, 137, ..., 148, 130,  99],
       ...,
       [ 18,  18,  27, ...,  73,  68,  62],
       [ 22,  22,  32, ...,  70,  71,  74],
       [ 22,  22,  32, ...,  70,  71,  74]], dtype=uint8), array([[226, 226, 223, ..., 230, 221, 200],
       [226, 226, 223, ..., 230, 221, 200],
       [226, 226, 223, ..., 230, 221, 200],
       ...,
       [ 84,  84,  92, ..., 173, 172, 177],
       [ 82,  82,  96, ..., 179, 181, 185],
       [ 82,  82,  96, ..., 179, 181, 185]], dtype=uint8)]


In [ ]:


import numpy as np 
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('D:\Mohamed\courses\Feed\OpenCV-in-Arabic-for-Beginners-master\images\lena.jpg',  cv2.IMREAD_GRAYSCALE)

hist = cv2.calcHist([img1], [0], None, [256],[0,256])
plt.plot(hist)



plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:

import numpy as np 
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('D:\Mohamed\courses\Feed\OpenCV-in-Arabic-for-Beginners-master\images\lena.jpg')

hist0 = cv2.calcHist([img1], [0], None, [256],[0,256])
hist1 = cv2.calcHist([img1], [1], None, [256],[0,256])
hist2 = cv2.calcHist([img1], [2], None, [256],[0,256])
hists = [hist0, hist1, hist2]
for i in range(3):
    plt.subplot(1,3,i+1), plt.plot(hists[i])
    


plt.show()

cv2.waitKey(0)
cv2.destroyAllWindows()